# Binary classification (Trauma detection)

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

GLB_INSTALL_DEPENDENCIES = False
GLB_USE_DRIVE_ACCOUNT = False

In [ ]:
# Here to define dependencies
if GLB_INSTALL_DEPENDENCIES:
    !pip install transformers
    !pip install torch
    !pip install openpyxl
"""
if GLB_USE_DRIVE_ACCOUNT:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive/"Colab Notebooks"
  username = "IsaacOlguin"
  repository =  "AutomatedTraumaDetectionInGCT"
  %cd {repository}
  %pwd
"""

In [ ]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
import json

import src.classification_model_utilities as mlclassif_utilities

## Globals

In [ ]:
PATH_DATASET = "input/dataset/20230112_Dataset_W_Batch1.xlsx" #"input/dataset/Dataset.xlsx"
PATH_DIR_LOGS = "logs/"
PATH_DIR_MODELS = "models/"
INDEX_COLUMNS_DATASET = 1
#LIST_NAME_COLUMNS_DATASET = ["id_document", "id_annotation", "span", "role", "trauma", "court"]
LIST_NAME_COLUMNS_DATASET = ["id_document", "id_annotation", "start", "span", "role", "trauma", "court", "Comments1", "Comments2"]

GLB_RETURN_ATTENTION_MASK = True
GLB_CROSS_VALIDATION = False
GLB_SAVE_MODEL = False
GLB_STORE_STATISTICS_MODEL = False
GLB_TEST_MODEL = False

# Globals for the model
EPOCHS = 3
EMBEDDING_SIZE = 512
NUM_CLASSES = 2
BATCH_SIZE = 8

## Dataset

In [ ]:
df_dataset = mlclassif_utilities.import_dataset_from_excel(PATH_DATASET, INDEX_COLUMNS_DATASET, LIST_NAME_COLUMNS_DATASET)
df_dataset.head()

In [ ]:
classes_dataset = mlclassif_utilities.get_unique_values_from_dataset(df_dataset, "trauma")
print(f"Num of different traumas in the dataset is {len(classes_dataset)} which are:")
for index, elem in enumerate(classes_dataset):
    print("\t", index+1, "-", elem)

### Distribution of all classes within the dataset

In [ ]:
distribution_classes = mlclassif_utilities.get_distribution_classes_from_dataset(df_dataset, ["trauma"], "span")

labels = list(distribution_classes["trauma"])
sizes = list(distribution_classes["span"])

fig1, ax1 = plt.subplots()
ax1.set_title(f'Distribution of classes in the dataset with a total number of {distribution_classes["span"].sum()} spans')
ax1.pie(sizes, labels=labels, autopct='%2.1f%%', shadow=True, startangle=0) #'%1.1f%%'
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

distribution_classes

# Train model

In [ ]:
!python /content/drive/MyDrive/"Colab Notebooks"/AutomatedTraumaDetectionInGCT/src/binary_classification.py